# Telegram bot

Reference: https://github.com/polakowo/vectorbt/blob/f78d1fd4ba4f70f91bbb0530fb29f338e2ceb41b/examples/TelegramSignals.ipynb#L172

In [3]:
# Prerequisites: at time being: please install python-telegram-bot lower than version 20, e.g., `pip install python-telegram-bot==13.*`
import os
import pandas as pd
import logging
from dotenv import load_dotenv, find_dotenv
from telegram.ext import CommandHandler
import vectorbt as vbt
from vectorbt.utils.datetime_ import get_utc_tz
import warnings

warnings.filterwarnings("ignore")

In [ ]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Initial Configuration
_ = load_dotenv(find_dotenv()) # load environment variable from .env file

# telegram
vbt.settings.messaging['telegram']['token'] = os.getenv("TELEGRAM_TOKEN")

# Giphy
vbt.settings.messaging['giphy']['api_key'] = os.getenv("GIPHY_API_KEY")
TZ_CONVERT = get_utc_tz()

In [ ]:
# Data
SYMBOLS = ["BTC-USD"]
START = '1 hour ago UTC'
TIMEFRAME = '1m' 
UPDATE_EVERY = vbt.utils.datetime_.interval_to_ms(TIMEFRAME) // 1000  # in seconds
DT_FORMAT = '%d %b %Y %H:%M:%S %z'
IND_PARAMS = dict(
    timeperiod=20, 
    nbdevup=2, 
    nbdevdn=2
)
CHANGE_NBDEV = 2

In [ ]:
data = vbt.YFData.download(SYMBOLS, start=START, interval=TIMEFRAME)

print(data.wrapper.index)

DatetimeIndex(['2023-08-12 07:07:00+00:00', '2023-08-12 07:08:00+00:00',
               '2023-08-12 07:09:00+00:00', '2023-08-12 07:10:00+00:00',
               '2023-08-12 07:11:00+00:00', '2023-08-12 07:12:00+00:00',
               '2023-08-12 07:13:00+00:00', '2023-08-12 07:14:00+00:00',
               '2023-08-12 07:15:00+00:00', '2023-08-12 07:16:00+00:00',
               '2023-08-12 07:17:00+00:00', '2023-08-12 07:18:00+00:00',
               '2023-08-12 07:19:00+00:00', '2023-08-12 07:20:00+00:00',
               '2023-08-12 07:21:00+00:00', '2023-08-12 07:22:00+00:00',
               '2023-08-12 07:23:00+00:00', '2023-08-12 07:24:00+00:00',
               '2023-08-12 07:25:00+00:00', '2023-08-12 07:26:00+00:00',
               '2023-08-12 07:27:00+00:00', '2023-08-12 07:28:00+00:00',
               '2023-08-12 07:29:00+00:00', '2023-08-12 07:30:00+00:00',
               '2023-08-12 07:31:00+00:00', '2023-08-12 07:32:00+00:00',
               '2023-08-12 07:33:00+00:00', '2023-0

In [ ]:
def get_bbands(data):
    return vbt.IndicatorFactory.from_talib('BBANDS').run(
        data.get('Close'), **IND_PARAMS, hide_params=list(IND_PARAMS.keys()))


def get_info(bbands):
    info = dict()
    info['last_price'] = bbands.close.iloc[-1]
    info['last_change'] = (bbands.close.iloc[-1] - bbands.close.iloc[-2]) / bbands.close.iloc[-1]
    info['last_crossed_above_upper'] = bbands.close_crossed_above(bbands.upperband).iloc[-1]
    info['last_crossed_below_upper'] = bbands.close_crossed_below(bbands.upperband).iloc[-1]
    info['last_crossed_below_lower'] = bbands.close_crossed_below(bbands.lowerband).iloc[-1]
    info['last_crossed_above_lower'] = bbands.close_crossed_above(bbands.lowerband).iloc[-1]
    info['bw'] = (bbands.upperband - bbands.lowerband) / bbands.middleband
    info['last_bw_zscore'] = info['bw'].vbt.zscore().iloc[-1]
    info['last_change_zscore'] = bbands.close.vbt.pct_change().vbt.zscore().iloc[-1]
    info['last_change_pos'] = info['last_change_zscore'] >= CHANGE_NBDEV
    info['last_change_neg'] = info['last_change_zscore'] <= -CHANGE_NBDEV
    return info


def format_symbol_info(symbol, info):
    last_change = info['last_change'][symbol]
    last_price = info['last_price'][symbol]
    last_bw_zscore = info['last_bw_zscore'][symbol]
    return "{} ({:.2%}, {}, {:.2f})".format(symbol, last_change, last_price, last_bw_zscore)


def format_signals_info(emoji, signals, info):
    symbols = signals.index[signals]
    symbol_msgs = []
    for symbol in symbols:
        symbol_msgs.append(format_symbol_info(symbol, info))
    return "{} {}".format(emoji, ', '.join(symbol_msgs))

In [ ]:
class MyTelegramBot(vbt.TelegramBot):
    def __init__(self, data, *args, **kwargs):
        super().__init__(data=data, *args, **kwargs)
        
        self.data = data
        self.update_ts = data.wrapper.index[-1]
        
    @property
    def custom_handlers(self):
        return (CommandHandler('info', self.info_callback),)
    
    def info_callback(self, update, context):
        chat_id = update.effective_chat.id
        if len(context.args) != 1:
            self.send_message(chat_id, "Please provide one symbol.")
            return
        symbol = context.args[0]
        if symbol not in SYMBOLS:
            self.send_message(chat_id, f"There is no such symbol as \"{symbol}\".")
            return
            
        bbands = get_bbands(self.data)
        info = get_info(bbands)
        messages = [format_symbol_info(symbol, info)]
        message = '\n'.join(["{}:".format(self.update_ts.strftime(DT_FORMAT))] + messages)
        self.send_message(chat_id, message)
        
    @property
    def start_message(self):
        index = self.data.wrapper.index
        return f"""Hello! 

Starting with {len(index)} rows from {index[0].strftime(DT_FORMAT)} to {index[-1].strftime(DT_FORMAT)}."""
        
    @property
    def help_message(self):
        return """Message format:
[event] [symbol] ([price change], [new price], [bandwidth z-score])
    
Event legend:
⬆️ - Price went above upper band
⤵️ - Price retraced below upper band
⬇️ - Price went below lower band
⤴️ - Price retraced above lower band

GIF is sent once a band is crossed and the price change is 2 stds from the mean."""

In [ ]:
telegram_bot = MyTelegramBot(data)
telegram_bot.start(in_background=True)

2023-08-12 16:06:39,098 - vectorbt.messaging.telegram - INFO - Initializing bot
2023-08-12 16:06:40,396 - vectorbt.messaging.telegram - INFO - Running bot ts_with_vectorbt_bot
2023-08-12 16:06:40,397 - apscheduler.scheduler - INFO - Scheduler started


In [ ]:
class MyDataUpdater(vbt.DataUpdater):
    def __init__(self, data, telegram_bot, **kwargs):
        super().__init__(data, telegram_bot=telegram_bot, **kwargs)
        
        self.telegram_bot = telegram_bot
        self.update_ts = data.wrapper.index[-1]
        
    def update(self):
        super().update()
        self.update_ts = pd.Timestamp.now(tz=TZ_CONVERT)
        self.telegram_bot.data = self.data
        self.telegram_bot.update_ts = self.update_ts
        
        bbands = get_bbands(self.data)
        info = get_info(bbands)
    
        messages = []
        if info['last_crossed_above_upper'].any():
            messages.append(format_signals_info('⬆️', info['last_crossed_above_upper'], info))
        if info['last_crossed_below_upper'].any():
            messages.append(format_signals_info('⤵️', info['last_crossed_below_upper'], info))
        if info['last_crossed_below_lower'].any():
            messages.append(format_signals_info('⬇️', info['last_crossed_below_lower'], info))
        if info['last_crossed_above_lower'].any():
            messages.append(format_signals_info('⤴️', info['last_crossed_above_lower'], info))
            
        if len(messages) > 0:
            message = '\n'.join(["{}:".format(self.update_ts.strftime(DT_FORMAT))] + messages)
            self.telegram_bot.send_message_to_all(message)
        if (info['last_crossed_above_upper'] & info['last_change_pos']).any():
            self.telegram_bot.send_giphy_to_all("launch")
        if (info['last_crossed_below_lower'] & info['last_change_neg']).any():
            self.telegram_bot.send_giphy_to_all("fall")

In [ ]:
data_updater = MyDataUpdater(data, telegram_bot)
data_updater.update_every(UPDATE_EVERY)

2023-08-12 16:06:40,685 - vectorbt.utils.schedule_ - INFO - Starting schedule manager with jobs [Every 60 seconds do update() (last run: [never], next run: 2023-08-12 16:07:40)]
2023-08-12 16:07:41,866 - vectorbt.data.updater - INFO - Updated data has 58 rows from 2023-08-12 07:07:00+00:00 to 2023-08-12 08:04:00+00:00
2023-08-12 16:08:46,796 - vectorbt.data.updater - INFO - Updated data has 59 rows from 2023-08-12 07:07:00+00:00 to 2023-08-12 08:05:00+00:00
2023-08-12 16:08:55,200 - vectorbt.messaging.telegram - INFO - 1004394120 - User: "/start"
2023-08-12 16:08:56,401 - vectorbt.messaging.telegram - INFO - 1004394120 - Bot: "Hello! 

Starting with 59 rows from 12 Aug 2023 07:07:00 +0000 to 12 Aug 2023 08:05:00 +0000."
2023-08-12 16:09:03,700 - vectorbt.messaging.telegram - INFO - 1004394120 - User: "/start"
2023-08-12 16:09:04,163 - vectorbt.messaging.telegram - INFO - 1004394120 - Bot: "Hello! 

Starting with 59 rows from 12 Aug 2023 07:07:00 +0000 to 12 Aug 2023 08:05:00 +0000."
20

In [ ]:
telegram_bot.stop()